Pay attention to the Andy's solution:
1) _validate_permission method is simple, exception is raised inside it.
2) role assignment and validation in _infer_permission is simple and straightforward.

As a result, there're no decorators and decorator properties. 

In [47]:
from enum import Flag, auto

In [48]:
class Permission(Flag):
    READ = auto()
    WRITE = auto()
    EXEC = auto()

In [3]:
Permission._value2member_map_

{1: <Permission.READ: 1>, 2: <Permission.WRITE: 2>, 4: <Permission.EXEC: 4>}

In [49]:
sum([permission.value for permission in Permission])

7

In [10]:
Permission(7)

<Permission.EXEC|WRITE|READ: 7>

In [127]:
import subprocess

In [176]:
class ValidatePermission:
    USER_GROUP = {
        "admin": Permission.READ | Permission.WRITE | Permission.EXEC,
        "user": Permission.READ,
        "manager": Permission.READ | Permission.WRITE,
        "support": Permission.EXEC
    }

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self

        return instance.__dict__.get(f"{type(self).__name__}_{self.name}")

    def __set__(self, instance, value):
        if not type(value) == int and not type(value) == str:
            raise TypeError("User permission must be an integer or a string-based user role")

        if type(value) == str and not value in self.USER_GROUP.keys():
            raise ValueError(f"User role must be one of the {list(self.USER_GROUP.keys())}")

        if type(value) == int:
            if value > sum([permission.value for permission in Permission]) or value < 1:
                instance.__dict__[f"{type(self).__name__}_{self.name}"] = Permission.READ
            else:
                instance.__dict__[f"{type(self).__name__}_{self.name}"] = Permission(value)

        if type(value) == str:
            instance.__dict__[f"{type(self).__name__}_{self.name}"] = self.USER_GROUP[value]

class BaseUser:
        # Better to refactor. Raise exception inside validation,and just call it from other methods
    def _validate_user_permission(self, required_permission):
        return required_permission in self.permissions

    def read(self, file):
            # Better to refactor, it's a redundant code
        if not self._validate_user_permission(Permission.READ):
            raise PermissionError(f"User doesn't have permission {Permission.READ.name} permission")
        
        try:
            with open(file) as script_file:
                contents = script_file.read()
        except FileNotFoundError:
            print(f"There's no file named {file}")
        else:
            print(contents)

    def write(self, file, content):
        if not self._validate_user_permission(Permission.WRITE):
            raise PermissionError(f"User doesn't have {Permission.WRITE.name} permission")
        
        with open(file, "w") as script_file:
            script_file.write(content)
        print(f"Wrote '{content}' to {file}.")

    def execute(self, file):
        if not self._validate_user_permission(Permission.EXEC):
            raise PermissionError(f"User doesn't have {Permission.EXEC.name} permission")
        
        subprocess.call(["python", file])

class User(BaseUser):

    permissions = ValidatePermission()

    def __init__(self, name, user_role) -> None:
        self.name = name
        self.permissions = user_role
        

In [177]:
u1 = User(name="Aaron", user_role="user")

In [178]:
u1.permissions

<Permission.READ: 1>

In [179]:
u2 = User("Andrew", "admin")

In [180]:
u2.permissions

<Permission.EXEC|WRITE|READ: 7>

In [193]:
u2.write(file="script.py", content="for i in range(10): print(i)")

Wrote 'for i in range(10): print(i)' to script.py.


In [194]:
u1.read('script.py')

for i in range(10): print(i)


In [195]:
u1.execute('script.py')

PermissionError: User doesn't have EXEC permission

In [196]:
u3 = User("Joseph", 6)

In [197]:
u3.permissions

<Permission.EXEC|WRITE: 6>

In [198]:
u3 = User("Joseph", 1236)

In [199]:
u3.permissions

<Permission.READ: 1>

In [200]:
u4 = User("Anna", "support")

In [201]:
u4.execute("script.py")

0
1
2
3
4
5
6
7
8
9


In [26]:
# Anya's advise: use decorators for checking permissions
# I decided to implement simplest solution first - just call internal _validate_user_permission,
# check if it returns True - go ahead with read/write/execute method, otherwise raise PermissionError
# Afterwards I can refactor as follows:
# - get rid of descriptor, create decorator property
# - implement decorator for _validate_user_permission

In [271]:

class BaseUser:

    USER_GROUP = {
        "admin": Permission.READ | Permission.WRITE | Permission.EXEC,
        "user": Permission.READ,
        "manager": Permission.READ | Permission.WRITE,
        "support": Permission.EXEC
    }

    @property
    def permissions(self):
        return self._permissions
    
    @permissions.setter
    def permissions(self, value):
        if not type(value) == int and not type(value) == str:
            raise TypeError("User permission must be an integer or a string-based user role")

        if type(value) == str and not value in self.USER_GROUP.keys():
            raise ValueError(f"User role must be one of the {list(self.USER_GROUP.keys())}")

        if type(value) == int:
            if value > sum([permission.value for permission in Permission]) or value < 1:
                self._permissions = Permission.READ
            else:
                self._permissions = Permission(value)

        if type(value) == str:
            self._permissions = self.USER_GROUP[value]

    def _validate_user_permission(required_permission):
        def decorator(func):
            def wrapper(self, *args, **kwargs):
                # print(self)
                # print(args, kwargs)
                if not required_permission in self.permissions:
                    raise PermissionError(f"User doesn't have permission {required_permission.name} permission")
                func(self, *args, **kwargs)
            return wrapper
        return decorator

    @_validate_user_permission(required_permission=Permission.READ)
    def read(self, file):  
        try:
            with open(file) as script_file:
                contents = script_file.read()
        except FileNotFoundError:
            print(f"There's no file named {file}")
        else:
            print(contents)

    @_validate_user_permission(required_permission=Permission.WRITE)
    def write(self, file, content):
        with open(file, "w") as script_file:
            script_file.write(content)
        print(f"Wrote '{content}' to {file}.")

    @_validate_user_permission(required_permission=Permission.EXEC)
    def execute(self, file):
        subprocess.call(["python", file])

    def __repr__(self) -> str:
        return f"{type(self).__name__}(name='{self.name}', user_role='{self.user_role}')"

class User(BaseUser):

    def __init__(self, name, user_role) -> None:
        self.name = name
        self.user_role = user_role
        self._permissions = None
        self.permissions = user_role


In [272]:
u1 = User(name="Aaron", user_role="user")

In [273]:
u1

User(name='Aaron', user_role='user')

In [258]:
u1.permissions

<Permission.READ: 1>

In [259]:
u2 = User("Andrew", "admin")

In [260]:
u2.permissions

<Permission.EXEC|WRITE|READ: 7>

In [261]:
u2.write(file="script.py", content="for i in range(10): print(i)")

Wrote 'for i in range(10): print(i)' to script.py.


In [262]:
u1.read('script.py')

for i in range(10): print(i)


In [263]:
u4 = User("Anna", "support")

In [264]:
u4.execute("script.py")

0
1
2
3
4
5
6
7
8
9


In [274]:
# Just to check Andy's calls to Permission flag

In [265]:
Permission(BaseUser.USER_GROUP.get("admin"))

<Permission.EXEC|WRITE|READ: 7>

In [266]:
BaseUser.USER_GROUP.get("admin")

<Permission.EXEC|WRITE|READ: 7>

In [275]:
type(BaseUser.USER_GROUP.get("admin"))

<enum 'Permission'>

In [267]:
Permission(Permission.READ | Permission.WRITE | Permission.EXEC)

<Permission.EXEC|WRITE|READ: 7>